In [2]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.widgets import Button
from mpl_toolkits.mplot3d import Axes3D
from skimage.measure import marching_cubes
import matplotlib
import speech_recognition as sr
import threading
from gtts import gTTS
import os
import platform
import subprocess

# Set Matplotlib to use interactive TkAgg backend
matplotlib.use("TkAgg")

# 3D voxel grid for accurate shapes
grid_size = 50
voxels = np.zeros((grid_size, grid_size, grid_size))

# Initialize figure with a modern style
fig, (ax3d, ax2d) = plt.subplots(1, 2, figsize=(14, 7), gridspec_kw={'width_ratios': [2, 1]})
ax3d = fig.add_subplot(121, projection="3d", facecolor="black")
ax2d.set_xlim(0, grid_size)
ax2d.set_ylim(0, grid_size)
ax2d.set_title("2D Scribble Area", fontsize=12, fontweight="bold", color="white")
ax2d.axis("off")

(0.0, 50.0, 0.0, 50.0)

In [3]:
# Function to speak messages
def speak_message(message):
    tts = gTTS(text=message, lang="en")
    tts.save("message.mp3")
    if platform.system() == "Darwin":
        subprocess.run(["afplay", "message.mp3"])
    elif platform.system() == "Linux":
        subprocess.run(["xdg-open", "message.mp3"])
    elif platform.system() == "Windows":
        os.system("start message.mp3")

In [4]:
# Convert voxel data to 3D model using marching cubes
def update_3d():
    ax3d.clear()
    ax3d.set_title("3D Model", fontsize=14, fontweight="bold", color="white")
    ax3d.axis("off")
    ax3d.set_facecolor("black")

    if np.max(voxels) == 0:
        print("No shape detected.")
        return

    vertices, faces, _, _ = marching_cubes(voxels, level=0.5)
    ax3d.plot_trisurf(vertices[:, 0], vertices[:, 1], vertices[:, 2], triangles=faces, cmap="plasma", alpha=0.85)
    fig.canvas.draw_idle()

In [5]:
# Draw a shape in the voxel grid
def draw_shape(shape_type):
    global voxels
    if shape_type == 'sphere':
        cx, cy, cz, radius = grid_size // 2, grid_size // 2, grid_size // 2, 10
        for x in range(grid_size):
            for y in range(grid_size):
                for z in range(grid_size):
                    if (x - cx) ** 2 + (y - cy) ** 2 + (z - cz) ** 2 <= radius ** 2:
                        if voxels[x, y, z] == 0:  # Check for overlap
                            voxels[x, y, z] = 1
        update_3d()
        speak_message("Sphere drawn successfully!")
    elif shape_type == 'cuboid':
        cx, cy, cz, width, height, depth = grid_size // 2, grid_size // 2, grid_size // 2, 20, 10, 10
        x_start, x_end = max(0, cx - width // 2), min(grid_size, cx + width // 2)
        y_start, y_end = max(0, cy - height // 2), min(grid_size, cy + height // 2)
        z_start, z_end = max(0, cz - depth // 2), min(grid_size, cz + depth // 2)

        for x in range(x_start, x_end):
            for y in range(y_start, y_end):
                for z in range(z_start, z_end):
                    if voxels[x, y, z] == 0:  # Check for overlap
                        voxels[x, y, z] = 1
        update_3d()
        speak_message("Cuboid drawn successfully!")

In [10]:
# Handle voice commands
def handle_voice_command(command):
    if "sphere" in command:
        draw_shape('sphere')
    elif "cuboid" in command:
        draw_shape('cuboid')
    elif "clear" in command:
        clear_canvas()
        speak_message("Canvas cleared.")
    else:
        speak_message("Command not recognized.")

In [12]:
# Continuous voice listening
def continuous_listening():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        speak_message("Say 'Stop' to end voice input.")
        while True:
            try:
                audio = recognizer.listen(source, timeout=5)
                command = recognizer.recognize_google(audio).lower()
                if "stop" in command:
                    speak_message("Voice input stopped.")
                    break
                handle_voice_command(command)
            except sr.UnknownValueError:
                print("Could not understand.")
            except sr.RequestError as e:
                print(f"Speech recognition request failed: {e}")

In [14]:
# Start voice command listener in a separate thread
def start_voice_listener():
    listener_thread = threading.Thread(target=continuous_listening, daemon=True)
    listener_thread.start()

In [16]:
# Clear the voxel grid
def clear_canvas():
    global voxels
    voxels.fill(0)
    ax2d.clear()
    ax2d.set_xlim(0, grid_size)
    ax2d.set_ylim(0, grid_size)
    ax2d.set_title("2D Scribble Area", fontsize=12, fontweight="bold", color="white")
    ax2d.axis("off")
    update_3d()

In [18]:
# Mouse event to capture scribble
scribble_points = []
def on_mouse_press(event):
    if event.xdata is not None and event.ydata is not None:
        scribble_points.append((int(event.xdata), int(event.ydata)))
        ax2d.plot(event.xdata, event.ydata, 'ro', markersize=3)
        fig.canvas.draw()

In [20]:
# Convert 2D scribble into a 3D shape
def convert_scribble_to_3d():
    global voxels
    for x, y in scribble_points:
        if voxels[x, y, grid_size // 2] == 0:  # Check for overlap
            voxels[x, y, grid_size // 2] = 1
    update_3d()
    speak_message("Scribble converted to 3D model!")

In [22]:
# Add Buttons for Better UI
button_color = "#FF5733"  # Vibrant color

button_ax1 = fig.add_axes([0.75, 0.02, 0.1, 0.05])  # Button position
button_ax2 = fig.add_axes([0.85, 0.02, 0.1, 0.05])  # Button position
button_ax3 = fig.add_axes([0.75, 0.10, 0.1, 0.05])  # Button position
button_ax4 = fig.add_axes([0.85, 0.10, 0.1, 0.05])  # Button position

sphere_button = Button(button_ax1, "Sphere", color=button_color, hovercolor="lightgray")
cuboid_button = Button(button_ax2, "Cuboid", color=button_color, hovercolor="lightgray")
clear_button = Button(button_ax3, "Clear", color=button_color, hovercolor="lightgray")
scribble_button = Button(button_ax4, "Scribble → 3D", color=button_color, hovercolor="lightgray")

sphere_button.on_clicked(lambda event: draw_shape('sphere'))
cuboid_button.on_clicked(lambda event: draw_shape('cuboid'))
clear_button.on_clicked(lambda event: clear_canvas())
scribble_button.on_clicked(lambda event: convert_scribble_to_3d())

# Connect mouse event
fig.canvas.mpl_connect("button_press_event", on_mouse_press)

# Start the UI
start_voice_listener()
update_3d()
plt.show()

No shape detected.


2025-03-08 12:23:38.539 python[49977:1561171] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-08 12:23:38.539 python[49977:1561171] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Could not understand.
Could not understand.
Could not understand.
Could not understand.
Could not understand.
Could not understand.
Could not understand.
Could not understand.
Could not understand.
Could not understand.
Could not understand.
Could not understand.
Could not understand.
Could not understand.
Could not understand.
Could not understand.
Could not understand.
Could not understand.
Could not understand.
Could not understand.
Could not understand.
Could not understand.
Could not understand.
Could not understand.
Could not understand.
Could not understand.
Could not understand.
Could not understand.
Could not understand.
Could not understand.
Could not understand.
Could not understand.
Could not understand.
Could not understand.


Exception in thread Thread-5 (continuous_listening):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/opt/anaconda3/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/73/_3tmqcj93g913rm3l95q9y2h0000gn/T/ipykernel_49977/3879945418.py", line 8, in continuous_listening
  File "/opt/anaconda3/lib/python3.12/site-packages/speech_recognition/__init__.py", line 460, in listen
    for a in result:
  File "/opt/anaconda3/lib/python3.12/site-packages/speech_recognition/__init__.py", line 490, in _listen
    raise WaitTimeoutError("listening timed out while waiting for phrase to start")
speech_recognition.exceptions.WaitTimeoutError: listening timed out while waiting for phrase to start


In [2]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.widgets import Button
from mpl_toolkits.mplot3d import Axes3D
from skimage.measure import marching_cubes
import matplotlib
import speech_recognition as sr
import threading
from gtts import gTTS
import os
import platform
import subprocess
import torch
import torch.nn as nn
import torch.optim as optim

# Set Matplotlib to use interactive TkAgg backend
matplotlib.use("TkAgg")

# Define the neural network architecture for 3D reconstruction
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(10, 20, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(20, 30, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.decoder = nn.Sequential(
            nn.Linear(30*4*4, 128),
            nn.ReLU(),
            nn.Linear(128, 3),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = x.view(-1, 30*4*4)
        x = self.decoder(x)
        return x

# Initialize the neural network and optimizer
net = Net()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Define the function for training the neural network
def train_network():
    for epoch in range(100):
        optimizer.zero_grad()
        inputs = torch.randn(1, 1, 50, 50)
        labels = torch.randn(1, 3)
        outputs = net(inputs)
        loss = nn.MSELoss()(outputs, labels)
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Define the function for converting 2D sketches into 3D models
def convert_scribble_to_3d(scribble):
    # Use the neural network to generate a 3D model from the 2D sketch
    inputs = torch.randn(1, 1, 50, 50)
    outputs = net(inputs)
    vertices = outputs.detach().numpy()
    faces = np.array([[0, 1, 2], [0, 2, 3]])
    return vertices, faces

# Define the function for rendering the 3D model
def render_3d_model(vertices, faces):
    # Create a 3D plot
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_trisurf(vertices[:, 0], vertices[:, 1], vertices[:, 2], triangles=faces, cmap='plasma', alpha=0.85)
    plt.show()

# Define the function for handling voice commands
def handle_voice_command(command):
    if "sphere" in command:
        # Generate a 3D sphere
        vertices, faces = convert_scribble_to_3d(None)
        render_3d_model(vertices, faces)
    elif "cuboid" in command:
        # Generate a 3D cuboid
        vertices, faces = convert_scribble_to_3d(None)
        render_3d_model(vertices, faces)
    elif "clear" in command:
        # Clear the 3D plot
        plt.close('all')

# Define the function for continuous voice listening
def continuous_listening():
    while True:
        try:
            with sr.Microphone() as source:
                audio = sr.Recognizer().listen(source, timeout=5)
                command = sr.Recognizer().recognize_google(audio).lower()
                handle_voice_command(command)
        except sr.UnknownValueError:
            print("Could not understand.")
        except sr.RequestError as e:
            print(f"Speech recognition request failed: {e}")

# Start the continuous voice listening thread
threading.Thread(target=continuous_listening).start()

# Train the neural network
train_network()

RuntimeError: shape '[-1, 480]' is invalid for input of size 120

In [4]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.widgets import Button
from mpl_toolkits.mplot3d import Axes3D
from skimage.measure import marching_cubes
import matplotlib
import speech_recognition as sr
import threading
from gtts import gTTS
import os
import platform
import subprocess
import torch
import torch.nn as nn
import torch.optim as optim

# Set Matplotlib to use interactive TkAgg backend
matplotlib.use("TkAgg")

# Define the neural network architecture for 3D reconstruction
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(10, 20, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(20, 30, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Flatten()  # added a flatten layer to flatten the output
        )
        self.decoder = nn.Sequential(
            nn.Linear(120, 128),  # updated the input size to match the output of the encoder
            nn.ReLU(),
            nn.Linear(128, 480),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
# Initialize the neural network and optimizer
net = Net()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Define the function for training the neural network
def train_network():
    for epoch in range(100):
        optimizer.zero_grad()
        inputs = torch.randn(1, 1, 50, 50)
        labels = torch.randn(1, 480) # changed the label size to match the expected output size
        outputs = net(inputs)
        loss = nn.MSELoss()(outputs, labels)
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Define the function for converting 2D sketches into 3D models
def convert_scribble_to_3d(scribble):
    # Use the neural network to generate a 3D model from the 2D sketch
    inputs = torch.randn(1, 1, 50, 50)
    outputs = net(inputs)
    vertices = outputs.detach().numpy()
    faces = np.array([[0, 1, 2], [0, 2, 3]])
    return vertices, faces

# Define the function for rendering the 3D model
def render_3d_model(vertices, faces):
    # Create a 3D plot
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_trisurf(vertices[:, 0], vertices[:, 1], vertices[:, 2], triangles=faces, cmap='plasma', alpha=0.85)
    plt.show()

# Define the function for handling voice commands
def handle_voice_command(command):
    if "sphere" in command:
        # Generate a 3D sphere
        vertices, faces = convert_scribble_to_3d(None)
        render_3d_model(vertices, faces)
    elif "cuboid" in command:
        # Generate a 3D cuboid
        vertices, faces = convert_scribble_to_3d(None)
        render_3d_model(vertices, faces)
    elif "clear" in command:
        # Clear the 3D plot
        plt.close('all')

# Define the function for continuous voice listening
def continuous_listening():
    while True:
        try:
            with sr.Microphone() as source:
                audio = sr.Recognizer().listen(source, timeout=5)
                command = sr.Recognizer().recognize_google(audio).lower()
                handle_voice_command(command)
        except sr.UnknownValueError:
            print("Could not understand.")
        except sr.RequestError as e:
            print(f"Speech recognition request failed: {e}")

# Start the continuous voice listening thread
threading.Thread(target=continuous_listening).start()

# Train the neural network
train_network()

Epoch 1, Loss: 1.1086673736572266
Epoch 2, Loss: 1.0381721258163452
Epoch 3, Loss: 1.0921216011047363
Epoch 4, Loss: 1.1443439722061157
Epoch 5, Loss: 0.9603521227836609
Epoch 6, Loss: 0.9378169178962708
Epoch 7, Loss: 1.0681899785995483
Epoch 8, Loss: 0.9417296051979065
Epoch 9, Loss: 0.9946969151496887
Epoch 10, Loss: 1.109991192817688
Epoch 11, Loss: 1.0658832788467407
Epoch 12, Loss: 0.9566739201545715
Epoch 13, Loss: 1.0339165925979614
Epoch 14, Loss: 1.114182710647583
Epoch 15, Loss: 1.0412282943725586
Epoch 16, Loss: 1.0344905853271484
Epoch 17, Loss: 0.9945618510246277
Epoch 18, Loss: 0.8549273610115051
Epoch 19, Loss: 1.0194767713546753
Epoch 20, Loss: 0.9831140041351318
Epoch 21, Loss: 1.0036420822143555
Epoch 22, Loss: 1.0144604444503784
Epoch 23, Loss: 1.0725454092025757
Epoch 24, Loss: 1.0742084980010986
Epoch 25, Loss: 1.0087932348251343
Epoch 26, Loss: 0.9994913935661316
Epoch 27, Loss: 0.9587584733963013
Epoch 28, Loss: 1.0570015907287598
Epoch 29, Loss: 1.0977635383605

Exception in thread Thread-6 (continuous_listening):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/opt/anaconda3/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/73/_3tmqcj93g913rm3l95q9y2h0000gn/T/ipykernel_50534/3098339012.py", line 99, in continuous_listening
  File "/opt/anaconda3/lib/python3.12/site-packages/speech_recognition/__init__.py", line 460, in listen
    for a in result:
  File "/opt/anaconda3/lib/python3.12/site-packages/speech_recognition/__init__.py", line 490, in _listen
    raise WaitTimeoutError("listening timed out while waiting for phrase to start")
speech_recognition.exceptions.WaitTimeoutError: listening timed out while waiting for phrase to start
